# Exercícios banco

Segue abaixo uma descrição das coleções a serem utilizadas nas tarefas seguintes:

Coleções presentes no banco de `analytics`:

**Banco principal**: `chatbot2`

* `establishments`: Coleção que armazena dos documentos dos estabelecimentos;
* `page`: Coleção que armazena os documentos relacionados a _page_ do estabelecimento;
* `messages`: Coleção que armazena as mensagens personalizadas do bot;
* `intentblocks`: Coleção que armazena as intenções do bot bloqueadas de serem respondidas por cada estabelecimento;
* `orders`: Coleção que armazena objetos de pedidos dos clientes dos estabelecimentos;
* `bills`: Coleção que armazena objetos referentes as faturas dos estabelecimentos no sistema;
* `establishmentunits`: Coleção que armazena dados das unidades do estabelecimento 
* `client_nps`: Coleção que armazena as respostas da pesquisa de NPS realizadas com clientes dos estabelecimentos
* `clientcoupons`: Coleção que armazena cupons retirados pelos clientes dos estabelecimentos
* `clientgifts`: Coleção que armazena os presentes ganhos pelos clientes no plano de fidelidade
* `modules`: Coleção que armazena os módulos do sistema
* `pagemodules`: Coleção que armazena os módulos que os estabelecimentos têm, e se eles estão ou não ativos assim como a data de contração.
* `item_prices`: Coleção que armazena os preços dos itens em cada dia da semana dos itens dos estabelecimentos
* `clients`: Coleção que armazena as informações dos clientes
## * `clientactions`: Coleção que armazena as ações dos clientes

Coleções presentes no banco `messages` (banco utilizado também para métricas:

**Banco principal**: `messages-bot-production`

* `messages`: Coleção que armazena as mensagens recebidas;


**Importante**: 
* Todas as operações que pedirem informações de um estabelecimento específico, deveremos utilizar a seguinte referência `page` para o objetivo: `ObjectId("5a0df4fc1e8bf00dcccccf7c")`
* Todos os resultados obtidos devem ser obtidos **somente utilizando consultas ao banco**
* Caso seja necessário, o repositório que guarda as *models* de cada uma das coleções está no seguinte link: https://github.com/githubanotaai/models
  

## Resgate todos os estabelecimentos ativos da plataforma 

Estabelecimentos ativos são aqueles que possuem valor `True` para a propriedade `sign.active`

In [ ]:
db.establishments.find({ "sign.active": true },{page:1, _id:0});   /* """ retorna todos os estabs """ */


## Resgate todos os estabelecimentos que entraram na plataforma antes de agosto de 2021

Podemos considerar a data de criação do objeto (somente neste exemplo) para induzir a data de entrada (`createdAt`)

In [ ]:
db.establishments.find({"createdAt": {"$lt": ISODate("2021-08-01")}},{createdAt:1, _id:0})


## Resgate quantos clientes tem o estabelecimento especificado

In [ ]:
db.clients.find({"page": ObjectId("5a0df4fc1e8bf00dcccccf7c")}).count()


## Resgate quantos clientes realizaram pedido no estabelecimento especificado


In [ ]:
db.orders.aggregate(
  {
    $match: { page: ObjectId("5a0df4fc1e8bf00dcccccf7c") },
  },
  {
    $group: {
      _id: "$page",
      count: { $sum: 1 },
    },
  }
);

6747;


## Resgate quais são as intenções bloqueadas do estabelecimento especificado

In [ ]:
db.intentsblocks.find(
    {page: ObjectId("5a0df4fc1e8bf00dcccccf7c")}, {whatsapp: 1, _id: 0, facebook: 1}
)




## Resgate quantos estabelecimentos possuem a intenção `observation` bloqueada independente do canal

In [ ]:
db.intentsblocks.find(
  { $or: [{ "whatsapp" : {$regex: "observation"} }, {"facebook" : {$regex: "observation"} }] }  
 ).count()

 54

## Resgate todos os estabelecimentos que tiveram valor de fatura superior a 200 reais no último mês

In [ ]:
var firstDayOfLastMonth=new Date();
firstDayOfLastMonth.setMonth(firstDayOfLastMonth.getMonth() - 1);
firstDayOfLastMonth.setDate(1);
var lastDayOfLastMonth=new Date();
lastDayOfLastMonth.setDate(1);
lastDayOfLastMonth.setDate(lastDayOfLastMonth.getDate() - 1);
db.bills.aggregate([
  {
    $match: 
    {
      $and: [{createdAt:{$gte:  firstDayOfLastMonth  }},{createdAt:{$lte:lastDayOfLastMonth}},{"totalBill": { $gt: 200 }}],      
    }
  },
  { $project: { _id: 1, createdAt: 1, totalBill:1 } } 
]);

## Resgate a quantidade de pedidos relativa a cada possível `check` _(status do pedido)_ do estabelecimento selecionado

* `check` é o campo que corresponde ao status do pedido;

Por exemplo, para cada valor de `check` possível, deveremos retornar a quantidade de pedidos dentro de cada um dos possíveis conjuntos.

In [ ]:
db.orders.aggregate(
  {
    $match: { page: ObjectId("5a0df4fc1e8bf00dcccccf7c") },
  },
  {
    $group: {
      _id: "$check",
      count: { $sum: 1 },
    },
  },
  {
    $sort: {_id:1}
  }
);

[
  { _id: 3, count: 5503 },
  { _id: 4, count: 154 },
  { _id: 5, count: 1024 },
  { _id: 6, count: 66 }
]

## Resgate a quantidade de mensagens existentes no banco que contém cada uma das palavras `pedido` e cardápio

O objetivo é saber quantas mensagens tiveram a palavra pedido e quantas tiveram a palavra cardápio

In [ ]:
sem acesso ao banco de msgs

## Resgate **somente** as informações de cidade e estado dos estabelecimentos ativos

Essas informações estão dentro de `establishmentunits`, dentro da propriedade `address`.


In [ ]:
db.establishments.aggregate([
  {
    $match: {
      "sign.active": true
    },
  },
  {
    $lookup: {
      from: "establishmentunits",
      foreignField: "page",
      localField: "page",
      as: "enderecos",
    },
  },
  { $project: {_id:0, page:1, "enderecos.address.city": 1, "enderecos.address.state": 1 } }
]);

## Resgate a quantidade de mensagens recebidas para cada intenção no desde o começo do dia de ontem

Observação: se a intenção foi igual a mensagem, a mesma deve ser desconsiderada

## Obtenha uma lista dos clientes de acordo com cada nota dada para o NPS abaixo de 7 desde o dia 1 de dezembro de 2021

In [ ]:
db.client_nps.aggregate([
  {
    $match: {
      
      $and: [
        { createdAt: { $gte: ISODate("2021-12-01") } },        
        { rating: { $lt: 7 } },
      ]
    },
  },
  {
    $lookup: {
      from: "clients",
      foreignField: "_id",
      localField: "client",
      as: "cliente",
    },
  },
  { $project: {_id:0, "cliente.name": 1, rating:1 } }
]);

## Resgate o total de cada uma notas NPS obtidas para cada mês desde agosto de 2021

In [ ]:
db.client_nps.aggregate([
  {
    $match: {
         createdAt: { $gte: ISODate("2021-08-01") }         
    }
  },  
  {
    $group: {
      _id: { 
        year: { $year: "$createdAt" }, 
        month: { $month: "$createdAt" } ,
        rating: "$rating"
      },      
      count: { $sum: 1 },      
    },
  },
  {
    $sort: {_id:1}
  }
]);

## Retorne as quantidades relativas a cada um dos métodos de pagamento utilizados pelos estabelecimentos

Quantos trabalham com cada tipo de método de pagamento?

Realize uma contagem dos tipos de cartão de crédito diferentes para sabermos qual é o mais utilizado

Observação: 

MASTERCARD, mastercard, MASTER CARD e master card deverão ser contabilizados como um só tipo


In [ ]:
db.establishmentunits.aggregate([
  {
    $match: {
      credit_cards: { $exists: true, $ne: [] }
    },
  },
  {
    $unwind: "$credit_cards"
  }, 
  {
    $group: {
      _id: "$credit_cards.name",
      count: { $sum: 1 }, 
    }
  },
  {
    $sort: { count: -1 },
  },
])

## Retorne os 10 estabelecimentos em que os clientes mais retiraram cupons

In [ ]:
db.clientcoupons.aggregate([    
  {
    $group: {
      _id: "$page",            
      count: { $sum: 1 },      
    },
  },
  {
     $sort: { count: -1 } 
  },
  {
    $limit: 10
  }
]);

## Retorne os 10 estabelecimentos em que os clientes mais retiraram prêmios do programa de fidelidade

Observação: Explore os documentos da coleção para descobrir quando um prêmio já foi retirado

In [ ]:

db.clientgifts.aggregate([ 
  {
    $match: {         
         $and: [
          {deleted: {$exists: false} },                      
          {date_withdrawal: {$exists: true} }
         ]     
    }
  }, 
  {
    $group: {
      _id: "$page",            
      count: { $sum: 1 },      
    },
  },
  {
     $sort: { count: -1 } 
  },
  {
    $limit: 10
  }
]);

## Retorne a quantidade de prêmios ainda a serem retirados por clientes de cada um dos estabelecimentos **com plano de fidelidade ativo**

In [33]:
//ta pegando os ativos

db.clientgifts.aggregate([
  {
    $match: {
      $and: [
        { deleted: { $exists: false } },
        { date_withdrawal: { $exists: false } },
      ],
    },
  },
  {
    $lookup: {
      from: "pagemodules",
      let: { page_from_clientgifts: "$page" },
      pipeline: [
        {
          $match: {
            $and: [
              { status: true },
              {$expr: {$eq: ["$module", ObjectId("5d948fa992326f56ca227b63")],},},
              { $expr: { $eq: ["$page", "$$page_from_clientgifts"] } },                
            ],
          },
        },
        
      ],
      as: "pages_with_fidelity_active",
    },
  },
  {
  $group: {
    _id:"$page",      
    count: { $sum: 1 }
  },
},
{
  $sort: { count: 1, _id:1 },
},
])

SyntaxError: invalid syntax (1057433801.py, line 3)

## Retorne todos os clientes que faltam apenas um ponto para que eles consigam ganhar o prêmio do plano fidelidade

Só deveremos considerar aqueles que possuem o plano de fidelidade ativo

In [ ]:
//nao entendi onde achar os pontos do fidelidade, pois no clientfidelities tem mais de um campo com withdrawn

## Retorne o cliente que mais realizou ações durante um atendimento no último dia

In [ ]:
db.clientactions.aggregate([
  {
    $match: 
      { "createdAt" : { $gte : ISODate("2022-06-14T00:00:00.000+0000"), $lt : ISODate("2022-06-15T00:00:00.000+0000") } }
    ,
  }, 
  
  {
    $group: {
      _id: "$client",
      count: { $sum: 1 }, 
    }
  },
  {
    $sort: { count: -1 },
  },
  {
    $limit:1
  }
 
])

## Retorne os 10 estabelecimentos que possuem mais itens **disponíveis** no cardápio

Quando um item não está disponível, a propriedade `deleted` dele está **setada** como `True`

In [ ]:
db.items.aggregate([
  {
    $match: 
      {deleted: false  }
    ,
  },  
  {
    $group: {
      _id: "$page",
      count: { $sum: 1 },
    },
  }, 
  {
    $sort: { count: -1 },
  },
  {
    $limit:10
  }
])

## Quais são os 5 estabelecimentos com maior quantidade de preços promocionais disponíveis?

Preços promocionais são aqueles que contém a propriedade `is_promotion` igual a `True`, dentro do preço incluso no objeto dos preços por dia da semana (`week`), na coleção `item_prices`

In [ ]:
db.item_prices.aggregate([
  {
    $match: 
    {
      $or: [
        { "week.0.is_promotion" : true },
        { "week.1.is_promotion" : true },
        { "week.2.is_promotion" : true },
        { "week.3.is_promotion" : true },
        { "week.4.is_promotion" : true },
        { "week.5.is_promotion" : true },
        { "week.6.is_promotion" : true }    
    ]
    }   
    ,
  },  
  {
    $group: {
      _id: "$page",
      count: { $sum: 1 },
    },
  }, 
  {
    $sort: { count: -1 },
  },
  {
    $limit:5
  }
])

## Qual a quantidade em meses que cada um dos estabelecimentos ativos estão na plataforma?

Para este exemplo (atenção, somente para este, no cenário real isso "não poderia" ser feito dessa forma), considere a data de entrada como a data de criação do documento de cada um dos estabelecimentos `createdAt`.

In [ ]:
db.establishments.aggregate([
  { $match: { "sign.active": true } },
  {
    $project: {
      
      years: {$round : [ {$divide:[{ $divide: [{ $subtract: ["$$NOW", "$createdAt"] }, 2629746000] },12]}, 0]},
      months:{$round : [    {$mod:[{ $divide: [{ $subtract: ["$$NOW", "$createdAt"] }, 2629746000] },12]}, 0]}
    }  
  },
  {
    $sort: { years: -1,months: -1 },
  },
]);

## Quais é a frase mais comum de cada uma das intenções do bot?

Observação: se a intenção foi igual a mensagem, esta mensagem deve ser desconsiderada
Observação: considerar apenas mensagens enviadas no dia de ontem

In [ ]:
nao da pra fazer sem a collection message

## Quantos estabelecimentos ativos temos em cada uma das regiões do país?

In [ ]:
db.establishmentunits.aggregate([
  {
    $match: {
      address: { $exists: true, $ne: [] },
    },
  },
    
  {
    $unwind: "$address.state",
  },
  {
    $group: {
      _id: "$address.state",
      count: { $sum: 1 },
    },
  },
  {
    $sort: { count: -1 },
  },
]);